In [15]:
from AlgorithmImports import *
import pandas as pd
import numpy as np
import sys
import os

from utils.indicator_wrapper import IndicatorWrapper, IndicatorManager
from utils.indicators import *
from utils.strategy import MLStrategy
from utils.labeller import Labeller, BaseStrategy
from utils.callbacks import CallbackHandler, SystemMonitorCallback, Callback
from sklearn.metrics import make_scorer
from skopt import BayesSearchCV
from functools import partial
from sklearn.model_selection import TimeSeriesSplit
from skopt.space import Real, Categorical, Integer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from datetime import timedelta

from tqdm import tqdm


ModuleNotFoundError: No module named 'numpy'

In [ ]:
import plotly.graph_objects as go
import random



def plot_label_context(
    df,
    label_col: str = "label",
    pre: int = 20,
    post: int = 20,
    max_events: int = 5
):
    res = []
    # estrai tutte le posizioni con label==1
    positions = [i for i, lab in enumerate(df[label_col]) if lab == 1]
 
    if not positions:
        print("Nessun evento label=1 da plottare.")
        return

    for i,pos in enumerate(positions):

        start = max(0, pos - pre)
        end   = min(len(df) - 1, pos + post)
        seg   = df.iloc[start : end+1]

        # asse X relativo: -pre…+post
        x = list(range(- (pos - start), len(seg) - (pos - start)))

        fig = go.Figure()
        fig.add_trace(go.Candlestick(
            x=x,
            open=seg["open"],
            high=seg["high"],
            low=seg["low"],
            close=seg["close"],
            name="Candles"
        ))
        fig.add_vline(x=0, line_dash="dash", line_color="red", line_width=2)
        fig.add_trace(go.Scatter(
            x=[0],
            y=[seg["close"].iloc[pos-start]],
            mode="markers",
            marker=dict(color="red", size=10),
            name="Event Close"
        ))
        fig.update_layout(
            title=f"Contesto label=1 (relativo) @ pos {pos}",
            xaxis_title="Bars from event",
            yaxis_title="Price",
            width=900, height=500
        )
        res.append(fig)
    return res

In [ ]:
from sklearn.metrics import fbeta_score, make_scorer

def group_nearby(indices, max_gap=1):
    """
    Raggruppa indici consecutivi o vicini (distanza <= max_gap).
    
    Args:
        indices (list or array): Lista ordinata di indici.
        max_gap (int): Massima distanza ammessa per considerare due elementi "vicini".
        
    Returns:
        List of lists: gruppi di indici.
    """
    if len(indices) == 0:
        return []

    groups = [[indices[0]]]
    for idx in indices[1:]:
        if idx - groups[-1][-1] <= max_gap:
            groups[-1].append(idx)
        else:
            groups.append([idx])
    return groups

def trading_scorer(y_val, y_pred):

    val_indices = np.where(y_val == 1)[0]
    pred_indices = np.where(y_pred == 1)[0]

    val_groups = group_nearby(val_indices)
    pred_groups = group_nearby(pred_indices)
    #print(f"pred_indices: {pred_groups}")
    #print(f"true_indices: {val_groups}")
    return fbeta_score(y_val, y_pred, beta=0.1)

In [ ]:
# Setup QuantBook
qb = QuantBook()

symbol = Symbol.Create("NVAX", SecurityType.Equity, Market.USA)
history = qb.History(symbol, timedelta(days=50), Resolution.Minute)

# Prepara il DataFrame nello stesso formato che usi nel main.py
history_df = pd.DataFrame({
    "open": history["open"],
    "high": history["high"],
    "low":  history["low"],
    "close": history["close"]
})
history_df.index

In [2]:
from QuantConnect.Indicators import (
    SimpleMovingAverage,
    ExponentialMovingAverage,
    AverageTrueRange,
    AverageDirectionalIndex,
    CommodityChannelIndex,
    Stochastic,
    OnBalanceVolume,
    MoneyFlowIndex,
    VolumeWeightedAveragePriceIndicator,
    DonchianChannel
)
from QuantConnect.Indicators import AverageTrueRange, MovingAverageType

ind_factories = {
    #"rsi":  lambda: IndicatorWrapper(RelativeStrengthIndex(14), "rsi", symbol),
    #"macd": lambda: IndicatorWrapper(MovingAverageConvergenceDivergence(12,26,9), "macd", symbol),
    #"bb":   lambda: IndicatorWrapper(BollingerBands(20, 2, MovingAverageType.Simple), "bb", symbol),
    "atr":  lambda: IndicatorWrapper(                       # NEW
            AverageTrueRange(14, MovingAverageType.Wilders),
            "atr",
            symbol
        ),
    #"ema_25":  lambda: IndicatorWrapper(PriceEmaDifferenceIndicator(25), "ema_25", symbol),
    #"ema_15":  lambda: IndicatorWrapper(PriceEmaDifferenceIndicator(15), "ema_15", symbol),
    #"ema_40":  lambda: IndicatorWrapper(PriceEmaDifferenceIndicator(40), "ema_40", symbol),
    #"ema_60":  lambda: IndicatorWrapper(PriceEmaDifferenceIndicator(60), "ema_60", symbol),
    #"slope": lambda: IndicatorWrapper(TrendlineSlopeIndicator(TrendlineFinder(lookback=100, kind="resistance", validations=2)), "slope", symbol),
    #"key_levels": lambda: IndicatorWrapper(KeyLevelsIndicator(kind="support",lookback=1000), "key_levels", symbol),
}

NameError: name 'os' is not defined

In [ ]:

from sklearn.pipeline import Pipeline
from sklearn.metrics import fbeta_score, make_scorer


spec_bull = BaseStrategy(
    id="long_ma20_fvg",
    bias="bullish",
    rules=[
        PriceEmaDifferenceRule(lookback=50,direction="negative", threshold=0.01),
        #BosIndicator(lookback=5000, levels_lag=10, breakout_lookback=10, locality=100, direction="bearish", key_level_validation=2),
        #TrendlineBreakoutIndicator(TrendlineFinder(lookback=200, kind="resistance")),
        KeyLevelBounceIndicator(lookback=500, levels_lag=10, bounce_lookback=10, locality=50, n_levels=3, min_touches=2,
        breakout=True, breakout_tol_pct=0.005),
        #InverseFVGRule(lookback=10, direction="bearish", body_multiplier=1.5),
        #FVGRule(lookback=3, direction="bullish", body_multiplier=1.5),
        #FvgBosRule(FVGRule(lookback=5, direction="bearish", body_multiplier=1.5),
        #BosIndicator(lookback=1000, levels_lag=10, breakout_lookback=5, locality=10, direction="bearish"))
        #TrendlineSlopeIndicator(TrendlineFinder()),
        #KeyLevelsIndicator(kind="support", lookback=1000, C = 0.001),
        #FVGRule(lookback=5, direction="bearish", body_multiplier=2.0),
        #BosIndicator(lookback=1000, levels_lag=10, bos_lookback=5, locality=20)s
    ]
)

"""rules=[
        PriceBelowMARule(ma_period=1, threshold_pct=0.01),
        FVGRule(lookback=150, must_retest=False, direction="bullish", body_multiplier=1.5)
    ]
"""
labeller = Labeller(spec_bull, lookahead=100, remove=False, sl=True, tpsl_ratio=1.0, slcoef=2.0)

strategy = MLStrategy(
    indicator_factories=ind_factories,
    labeller=labeller,
    id="long_ma20_fvg",
)

 # Hyperparameter optimization
param_space = {
    "classifier__scale_pos_weight": Real(0.1, 0.6),
    "classifier__n_estimators": Integer(1000, 1500),
    "classifier__max_depth": Integer(2, 9),
    "classifier__learning_rate": Real(0.001, 0.01),
    "classifier__min_child_weight": Integer(1, 3),
}
callbacks = [SystemMonitorCallback()]
classifier = XGBClassifier(base_score=0.5)


pipeline = Pipeline([
    ("classifier", classifier)
])
scorer = make_scorer(trading_scorer, greater_is_better=True)
search = BayesSearchCV(
                estimator=pipeline,
                scoring=scorer,
                search_spaces=param_space,
                cv=TimeSeriesSplit(n_splits=3),
                verbose=1,
                n_jobs=-1,
                n_iter=1,
                error_score=np.nan
            )
res = strategy.model_selection(
    train_df=history_df,
    model=search,
    callbacks=callbacks
)
percent_ones = res["labels"].sum() / len(res["train_df"]) * 100
print(res["labels"].sum())

print(f"Percentuale di 1 nelle etichette: {percent_ones:.2f}%")
print(res["best_score"])

figs = plot_label_context(
    res["train_df"].assign(label=res["labels"].values),
    label_col="label",
    pre=500,
    post=100,
    max_events=1
)


In [ ]:
from IPython.display import display
for fig in figs[-10:]:
    display(fig)